In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 122)

### 【問題1】コンペティション内容の把握
コンペティションのOverviewページ読み、「Home Credit Default Risk」はどのようなコンペティションか、\
以下の観点からレポートしてください。


- 1. Home Creditはどのような企業？
- 2. このコンペティションでは何を予測する？
- 3. それを予測することで企業が得られるメリットは何？

- 1. Home Creditはどのような企業？

Founded in 1997, Home Credit is an international consumer finance provider with operations in 9 countries.
We focus on responsible lending primarily to people with little or no credit history.
Our services are simple, easy and fast.

1997年に設立されたHome Creditは９カ国で運営されている国際的な消費者金融会社です。当社は、信用状況報告の無い、又は、少ない人向けの融資に焦点を当てています。当社のサービスはシンプルで簡単、そして迅速です。

- 2.このコンペティションでは何を予測する？

返済能力の予測

- 3.それを予測することで企業が得られるメリットは何？

貸し倒れのリスクの回避など

### 【問題2】データの概観の把握
データの概観を把握するために以下のようなことを行ってください。


- .head()、.info()、describe()などを使う
- 欠損値の有無を見る
- クラスごとのデータ数をグラフに出す

それぞれ結果に対する説明や考察も行ってください。

In [ ]:
app_train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')

In [ ]:
app_train.head()

In [ ]:
app_train.info()

In [ ]:
app_train.describe()

In [ ]:
app_train.shape

In [ ]:
app_train.isnull().any()

欠損値の外観を把握

In [ ]:
msno.matrix(app_train)

欠損値の割合を把握

In [ ]:
pd.options.display.float_format = '{: <10.2%}'.format
total = app_train.isnull().sum().sort_values(ascending=False)
percent = (app_train.isnull().sum()/app_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(sum(total > 0))

In [ ]:
pd.reset_option('display.float_format')

目的変数('TARGET')の要素の頻度及びカウントプロットを表示

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
sns.countplot(x='TARGET', data=app_train)

### 【問題3】課題設定
データの概観を見たことを元に、自分なりの課題・疑問を複数設定してください。

《相関の強さ》\
\begin{eqnarray}
0.7 ～ 1.0　&=& \mbox{かなり強い正の相関がある}\\
0.4 ～ 0.7　&=& \mbox{正の相関がある}\\
0.2 ～ 0.4　&=& \mbox{弱い正の相関がある}\\
－0.2 ～ 0 ～ 0.2　&=& \mbox{ほとんど相関がない}\\
－0.4 ～ －0.2　&=& \mbox{弱い負の相関がある}\\
－0.7 ～ －0.4　&=& \mbox{負の相関がある}\\
－1.0 ～ －0.7　&=& \mbox{かなり強い負の相関がある}\\
\end{eqnarray}

- TARGET変数と特徴量間で「ほとんど相関がない」
- 相関性に焦点を当てたデータ探索が困難
- コンペでは「返済能力の予測」が課題
- どのような人であれば融資するかを検討
 1. 年齢
     1. 年齢の増加と収入・金融資産の増加の関係性
 2. 収入
 3. 職業
 4. 性別
 5. 家族
 6. 住宅・車両・金融資産の保有状況(担保になりうる)
 7. 学歴
 8. ローン完済年

### 【問題4】データ探索
問題3で設定した課題・疑問を解決するためにデータ探索を進めます。考察を伴う表やグラフを 15個以上 作成してください。


探索過程で新しい疑問点が生まれた場合は、問題3の回答に加えていってください。

121列ある特徴量を約40列ごとに分割して、TARGET変数と結合した上で相関性を把握

In [ ]:
features = app_train.loc[:, app_train.columns != 'TARGET']
target = app_train.loc[:, app_train.columns == 'TARGET']

features_corr_1 = app_train.iloc[:, :40]
features_corr_2 = app_train.iloc[:, 40:80]
features_corr_3 = app_train.iloc[:, 80:121]

features_corr_1 = pd.concat([target, features_corr_1], axis=1)
features_corr_2 = pd.concat([target, features_corr_2], axis=1)
features_corr_3 = pd.concat([target, features_corr_3], axis=1)

In [ ]:
plt.figure(figsize=(30, 24)) 
sns.heatmap(features_corr_1.corr(), annot=True, square=True, fmt='.2f', cmap='gist_rainbow', vmin=-1, vmax=1)
plt.show()

In [ ]:
plt.figure(figsize=(30, 24)) 
sns.heatmap(features_corr_2.corr(), annot=True, square=True, fmt='.2f', cmap='gist_rainbow', vmin=-1, vmax=1)
plt.show()

In [ ]:
plt.figure(figsize=(30, 24)) 
sns.heatmap(features_corr_3.corr(), annot=True, square=True, fmt='.2f', cmap='gist_rainbow', vmin=-1, vmax=1)
plt.show()

- __<u>年齢</u>__

In [ ]:
app_train['DAYS_BIRTH_YEAR'] = (app_train['DAYS_BIRTH'] // 365).abs()

age_ctgr_list = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
age_ctgr_name = ['20-25', '25-30', '30-35', '35-40', '40-45', '45-50', '50-55', '55-60', '60-65', '65-70']
 
app_train['DAYS_BIRTH_CTGR'] = pd.cut(app_train['DAYS_BIRTH_YEAR'], bins=age_ctgr_list, labels=age_ctgr_name)

In [ ]:
sns.countplot(x='DAYS_BIRTH_CTGR', hue='TARGET', data=app_train)

年齢を５歳毎に区切り、TARGET別のデータ件数を確認

In [ ]:
sns.kdeplot(app_train['DAYS_BIRTH_YEAR'][app_train['TARGET'] == 0], shade=True, color="b", label='TARGET = 0')
sns.kdeplot(app_train['DAYS_BIRTH_YEAR'][app_train['TARGET'] == 1], shade=True, color="r", label='TARGET = 1')

1. 若年層に返済が困難になる者が多い傾向あり
2. 年齢が上がるにつれて返済が困難になる者が減少する傾向あり
3. 上記2.の要因として、年齢の増加と収入・金融資産の増加の関係性を指摘する

- __<u>収入</u>__

1. 'AMT_INCOME_TOTAL'は、年収を管理する列になっていないようである(年収にしては高額)。
2. 'AMT_INCOME_TOTAL'を'AMT_CREDIT'や''DAYS_BIRTH'で割り、TARGET変数との関連性を探る。

In [ ]:
app_train['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'] = app_train['AMT_INCOME_TOTAL'] / app_train['DAYS_BIRTH'].abs()
app_train['AMT_INCOME_TOTAL_IN_AMT_CREDIT'] = app_train['AMT_INCOME_TOTAL'] / app_train['AMT_CREDIT']

In [ ]:
app_train['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'].describe()

In [ ]:
income_analysis = pd.concat([target, app_train['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'],app_train['AMT_INCOME_TOTAL_IN_AMT_CREDIT']], axis=1)

In [ ]:
#外れ値の削除(改善余地あり)
for inx in income_analysis['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'][income_analysis['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'] > 26].index:
    income_analysis.drop(index=inx, inplace=True)

In [ ]:
grid = sns.FacetGrid(income_analysis, col='TARGET', hue='TARGET', col_wrap=2)
grid.map(sns.distplot, 'AMT_INCOME_TOTAL_IN_DAYS_BIRTH', bins=10, kde=False)
plt.show()

In [ ]:
sns.kdeplot(income_analysis['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'][income_analysis['TARGET'] == 0], shade=True, color="b", label='TARGET = 0')
sns.kdeplot(income_analysis['AMT_INCOME_TOTAL_IN_DAYS_BIRTH'][income_analysis['TARGET'] == 1], shade=True, color="r", label='TARGET = 1')

In [ ]:
#外れ値の削除(改善余地あり)
for inx in income_analysis['AMT_INCOME_TOTAL_IN_AMT_CREDIT'][income_analysis['AMT_INCOME_TOTAL_IN_AMT_CREDIT'] > 1].index:
    income_analysis.drop(index=inx, inplace=True)

In [ ]:
grid = sns.FacetGrid(income_analysis, col='TARGET', hue='TARGET', col_wrap=2)
grid.map(sns.distplot, 'AMT_INCOME_TOTAL_IN_AMT_CREDIT', bins=10, kde=False)
plt.show()

In [ ]:
sns.kdeplot(income_analysis['AMT_INCOME_TOTAL_IN_AMT_CREDIT'][income_analysis['TARGET'] == 0], shade=True, color="b", label='TARGET = 0')
sns.kdeplot(income_analysis['AMT_INCOME_TOTAL_IN_AMT_CREDIT'][income_analysis['TARGET'] == 1], shade=True, color="r", label='TARGET = 1')

収入の当該指標値の高低差と返済の困難性は余り関係がなさそうである。

- __<u>職業</u>__

In [ ]:
pd.DataFrame(app_train.groupby(['OCCUPATION_TYPE', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.catplot(y='OCCUPATION_TYPE', hue='TARGET', data=app_train, kind="count", height=8, aspect=1)

クライアント職業分布
1. 労働者(Laborers)
2. 販売員(Sales staff)
3. コアスタッフ(Core staff)
4. マネージャー(Managers)
5. 運転手(Drivers)

- __<u>性別</u>__

In [ ]:
pd.DataFrame(app_train.groupby(['CODE_GENDER', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(x='CODE_GENDER', hue='TARGET', data=app_train)

男性は女性と比較して、返済が困難になる者が多い傾向あり

- __<u>家族</u>__

1. 婚姻状況

In [ ]:
pd.DataFrame(app_train.groupby(['NAME_FAMILY_STATUS', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(y='NAME_FAMILY_STATUS', hue='TARGET', data=app_train)

結婚・独身などで返済の困難性に違いは見受けられないようである。

2. 子供の有無

In [ ]:
pd.DataFrame(app_train.groupby(['CNT_CHILDREN', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(y='CNT_CHILDREN', hue='TARGET', data=app_train)

子供の数で返済の困難性に違いは見受けられないようである。

3. 申請時の同行者

In [ ]:
pd.DataFrame(app_train.groupby(['NAME_TYPE_SUITE', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(y='NAME_TYPE_SUITE', hue='TARGET', data=app_train)

- __<u>車・不動産の所有状況</u>__

1. 車の所有状況

In [ ]:
pd.DataFrame(app_train.groupby(['FLAG_OWN_CAR', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(x='FLAG_OWN_CAR', hue='TARGET', data=app_train)

2. 不動産の所有状況

In [ ]:
pd.DataFrame(app_train.groupby(['FLAG_OWN_REALTY', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(x='FLAG_OWN_REALTY', hue='TARGET', data=app_train)

3. 住宅のタイプ

In [ ]:
pd.DataFrame(app_train.groupby(['NAME_HOUSING_TYPE', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(y='NAME_HOUSING_TYPE', hue='TARGET', data=app_train)

- __<u>学歴</u>__

In [ ]:
pd.DataFrame(app_train.groupby(['NAME_EDUCATION_TYPE', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
sns.countplot(y='NAME_EDUCATION_TYPE', hue='TARGET', data=app_train)

データ数こそ多くないものの学位(Academic degree)取得者が返済困難になるケースが少ない傾向あり

- __<u>ローン完済年</u>__

In [ ]:
app_train['AMT_CREDIT_PAYOFF_YEAR'] = app_train['AMT_CREDIT'] // app_train['AMT_ANNUITY']

In [ ]:
pd.DataFrame(app_train.groupby(['AMT_CREDIT_PAYOFF_YEAR', 'TARGET'])['SK_ID_CURR'].count())

In [ ]:
grid = sns.FacetGrid(app_train, col='TARGET', hue='TARGET', col_wrap=2)
grid.map(sns.distplot, 'AMT_CREDIT_PAYOFF_YEAR', bins=20, kde=False)
plt.show()

In [ ]:
sns.kdeplot(app_train['AMT_CREDIT_PAYOFF_YEAR'][app_train['TARGET'] == 0], shade=True, color="b", label='TARGET = 0')
sns.kdeplot(app_train['AMT_CREDIT_PAYOFF_YEAR'][app_train['TARGET'] == 1], shade=True, color="r", label='TARGET = 1')

完済年が長い場合に返済の困難性が高くなる傾向を想定したが、傾向性は見られないようである。

### 【問題5】（アドバンス課題）Notebooksへの投稿
必須ではありませんが、コンペティションのNotebooksページに今回作成したEDAのノートブックを投稿することを推奨しています。